# Data Import, Cleaning, and Preparation

This module is used to query the postgreSQL database in order to retrieve the Yelp and Violations dataset. There will be several steps to do this:
* Import flattened violations dataset. We will use a SQL query to perform aggregation of violation data by restaurant and inspeciton date. 
* Aggregate the Yelp Data
    * Reviews for a given establishment will be aggregated so that reviews *after* the previous inspection (or the earliest review date) and *before* the date of a given inspection are in one batch. 
    * Aggregate "count" features using this same logic
    * Combine the review document for a restaurant into a CLOB using the same logic
* LEFT JOIN the violations dataset to the Yelp data after aggregationis complete for both datasets

In [60]:
import psycopg2 as psy
import pandas as pd
import re
import numpy as np

In [61]:
#set up connection to our DB
conn = psy.connect(database="sterndsyelp", 
                        user="mvsternds", 
                        password="nyustern123!", 
                        host="sterndsyelp.cawzspvmqd5q.us-east-1.rds.amazonaws.com", 
                        port="5432"
                       )
#open cursor and check our tables in the DB
cur = conn.cursor()
cur.execute("SELECT table_name FROM information_schema.tables"
            " WHERE table_schema='public'" 
            " AND table_type='BASE TABLE'")
rows =cur.fetchall()
print(pd.DataFrame(rows))

                       0
0               business
1                checkin
2             trnt_insps
3  violations_pittsburgh
4                   tips
5                reviews


In [62]:
cur.execute("SELECT * FROM public.restaurants LIMIT 50")
biz = pd.DataFrame(cur.fetchall())

cur.execute("SELECT * FROM public.checkin LIMIT 50")
checkins = pd.DataFrame(cur.fetchall())

cur.execute("SELECT * FROM public.v_reviews LIMIT 50")
reviews = pd.DataFrame(cur.fetchall())

**NOTE: ONLY LIMITING to 50 rows during build phase to limit processing time. **

In [63]:
biz.columns = ['bizID', 'name', 'address', 'zip', 'neighborhood', 'lat','long', 'categories','attributes','is_open','review_count','hours','stars']
biz.describe()

,bizID,name,address,zip,neighborhood,lat,long,categories,attributes,is_open,review_count,hours,stars
count,50,50,50,50,50,50,50,50,50,50,50,50,50
unique,50,48,48,48,22,49,49,48,50,2,23,36,8
top,ghWwqlRd1MaWz8hfgZ8Vsw,Subway,,,,43.7898425,-79.4180592,"['Restaurants', 'Pizza']","['Alcohol: full_bar', ""Ambience: {'romantic': ...",1,7,,3.0
freq,1,2,2,2,14,2,2,3,1,39,8,15,17


In [64]:
reviews.columns = ['bizID','reviewID','userID','type','stars','text','useful','funny','cool','date']
#get dummies for star rating column
reviews = pd.concat([reviews, pd.get_dummies(reviews['stars'], prefix='stars')], axis=1)
reviews.head()

,bizID,reviewID,userID,type,stars,text,useful,funny,cool,date,stars_1,stars_2,stars_3,stars_4,stars_5
0,--SrzpvFLwP_YFwB_Cetow,55o8orbs6LoOjJU7PXNObQ,2vJ2e51kdbdAmAo_HTr4KQ,review,4,"""Out of all the Keung Kee locations, my family...",2,0,2,2011-06-08,0,0,0,1,0
1,--SrzpvFLwP_YFwB_Cetow,9n93CvomhMGj5Nrt6NcppQ,3qMeM2IZP1L_E-aI08-0xw,review,1,Cockroach in my congee. Mediocre service. Medi...,3,1,0,2015-02-23,1,0,0,0,0
2,--SrzpvFLwP_YFwB_Cetow,aR0BONm-RsvkMW_kyy80Ew,lr-uNDv2NMIURR8WT9Z2GQ,review,4,"""This place is a great Chinese restaurant for ...",0,0,0,2012-12-28,0,0,0,1,0
3,--SrzpvFLwP_YFwB_Cetow,n7eKfZdyp6uE8PkAz27IEg,EiP1OFgs-XGcKZux0OKWIA,review,3,We came for a regular dinner. We ordered a Cl...,0,1,1,2015-01-13,0,0,1,0,0
4,--SrzpvFLwP_YFwB_Cetow,7R2vvkJCE918Q8ORegPrPQ,2Z0I9POv2N5p1_vOnCN9lw,review,5,Great for noodle and rice dishes. Done really ...,0,0,0,2013-06-28,0,0,0,0,1


In [65]:
checkins.columns = ['bizID','type','datetime']
checkins.describe()

,bizID,type,datetime
count,50,50,50
unique,50,1,50
top,b'N6VRRM1XZ6c92WBhWWu1XQ',b'checkin',"['Fri-0:1', 'Sat-14:1', 'Mon-15:2', 'Thu-15:1'..."
freq,1,50,1


In [66]:
#get list of unique biz and create df
#uniquebiz = []
#for x in reviews['bizID']:
    #if x not in uniquebiz:
        #uniquebiz.append(x)

uniquebiz=np.unique(reviews['bizID'])
bizrevs = pd.DataFrame(uniquebiz)
bizrevs.columns = ['bizID']
bizrevs = pd.merge(bizrevs,biz[['bizID','name','address']],on='bizID', how='left')
bizrevs.head()

,bizID,name,address
0,--SrzpvFLwP_YFwB_Cetow,NaN,NaN
1,0_MI3uzOP0MMPNSfj5zB-g,NaN,NaN
2,4HeS4zlbf-IbwEItbrISlw,NaN,NaN
3,9iIztpy5MJuSFrLgK1HIcw,NaN,NaN
4,Cfj526DoysI9UvojIrRsUw,NaN,NaN


**Note: merge in cell above won't result in matches for name/address because of 50 row limit put in place. should work after removing limit**

In [67]:
#declare rest of columns
bizrevs['reviews'] = ""
bizrevs['checkins'] = 0
bizrevs['stars_1'] = 0
bizrevs['stars_2'] = 0
bizrevs['stars_3'] = 0
bizrevs['stars_4'] = 0
bizrevs['stars_5'] = 0
bizrevs['reviews_whole_words'] = ""
bizrevs.head()

,bizID,name,address,reviews,checkins,stars_1,stars_2,stars_3,stars_4,stars_5,reviews_whole_words
0,--SrzpvFLwP_YFwB_Cetow,NaN,NaN,,0,0,0,0,0,0,
1,0_MI3uzOP0MMPNSfj5zB-g,NaN,NaN,,0,0,0,0,0,0,
2,4HeS4zlbf-IbwEItbrISlw,NaN,NaN,,0,0,0,0,0,0,
3,9iIztpy5MJuSFrLgK1HIcw,NaN,NaN,,0,0,0,0,0,0,
4,Cfj526DoysI9UvojIrRsUw,NaN,NaN,,0,0,0,0,0,0,


In [68]:
#look up each unique biz ID in reviews table
for i in range(len(bizrevs['bizID'])):
    #add text of review to reviews column if biz IDs match
    for x in range(len(reviews['bizID'])):
        if bizrevs['bizID'][i] == reviews['bizID'][x]:
            bizrevs['reviews'][i] = bizrevs['reviews'][i] + reviews['text'][x]
    #count number of checkins
    for y in range(len(checkins['bizID'])):
        if bizrevs['bizID'][i] == checkins['bizID'][y]:
            bizrevs['checkins'][i] = bizrevs['checkins'][i] + 1
    #count number of reviews with each star rating
    for z in range(len(reviews['bizID'])):
        if bizrevs['bizID'][i] == reviews['bizID'][z]:
            bizrevs['stars_1'][i] = bizrevs['stars_1'][i] + reviews['stars_1'][z]
            bizrevs['stars_2'][i] = bizrevs['stars_2'][i] + reviews['stars_2'][z]
            bizrevs['stars_3'][i] = bizrevs['stars_3'][i] + reviews['stars_3'][z]
            bizrevs['stars_4'][i] = bizrevs['stars_4'][i] + reviews['stars_4'][z]
            bizrevs['stars_5'][i] = bizrevs['stars_5'][i] + reviews['stars_5'][z]
    #extract whole words from reviews
    bizrevs['reviews_whole_words'][i] = ' '.join(re.findall('[A-Za-z]+', bizrevs['reviews'][i]))
    
bizrevs.head()

/Users/cmvoelker/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/cmvoelker/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/cmvoelker/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/Users/cmvoelker/anaconda/lib/python3.6/site-packages/ipykernel_launcher

,bizID,name,address,reviews,checkins,stars_1,stars_2,stars_3,stars_4,stars_5,reviews_whole_words
0,--SrzpvFLwP_YFwB_Cetow,NaN,NaN,"""Out of all the Keung Kee locations, my family...",0,1,2,4,10,4,Out of all the Keung Kee locations my family p...
1,0_MI3uzOP0MMPNSfj5zB-g,NaN,NaN,"""Can't say they've kept up the good work. Pri...",0,1,1,0,0,0,Can t say they ve kept up the good work Prices...
2,4HeS4zlbf-IbwEItbrISlw,NaN,NaN,"""This is a welcome addition to the neighborhoo...",0,2,1,0,6,2,This is a welcome addition to the neighborhood...
3,9iIztpy5MJuSFrLgK1HIcw,NaN,NaN,"""I'll never understand the draw to Tim Horton'...",0,0,2,2,2,0,I ll never understand the draw to Tim Horton s...
4,Cfj526DoysI9UvojIrRsUw,NaN,NaN,"""$3.99 all day specials.. no judgement please....",0,0,1,2,0,0,all day specials no judgement please n nEvery ...


In [69]:
#import toronto inspection data
cur.execute("SELECT * FROM public.toronto_inspections LIMIT 50")
insp = pd.DataFrame(cur.fetchall())
insp.columns = ['insp_bizID','insp_biz_name','insp_biz_address','insp_date', 'last_inspection','insp_count_minor','insp_count_significant','insp_count_crucial','insp_count_na','insp_total_count_cs']
insp.head()

,insp_bizID,insp_biz_name,insp_biz_address,insp_date,last_inspection,insp_count_minor,insp_count_significant,insp_count_crucial,insp_count_na,insp_total_count_cs
0,10187087,METROPOLIS BAKESHOP,2 BLOOR ST W,3/17/17,10/18/16,0,0,0,0,0
1,10289695,MILAN'S PIZZERIA & WINGS,1792 WESTON RD,11/9/15,11/2/15,0,0,0,0,0
2,10355463,PIZZA PASTA WAYS,3300 BLOOR ST W,12/15/16,10/7/15,2,0,0,0,0
3,10453689,TRINITY - ST. PAUL'S UNITED CHURCH,427 BLOOR ST W,2/18/16,12/21/15,0,0,0,0,0
4,10503134,INTERNATIONAL NEWS SHEPPARD,45 SHEPPARD AVE E,1/31/17,1/31/17,1,0,0,0,0


In [70]:
#declare function to calculate levenshtein distance between 2 strings (not case sensitive)
def lev(string1, string2):
    #delete the ".lower()" in the following two lines to make distance case sensitive
    s1=str.strip(string1.lower())
    s2=str.strip(string2.lower())
    m=len(s1)+1
    n=len(s2)+1

    tbl = {}
    for i in range(m): tbl[i,0]=i
    for j in range(n): tbl[0,j]=j
    for i in range(1, m):
        for j in range(1, n):
            cost = 0 if s1[i-1] == s2[j-1] else 1
            tbl[i,j] = min(tbl[i, j-1]+1, tbl[i-1, j]+1, tbl[i-1, j-1]+cost)

    return tbl[i,j]

#test the function
print(lev('Hello',"hello"))
print(lev('dock','duck '))
print(lev('st','saint'))

0
1
3


**note: next cell should return matches once we include more than the 50 rows (fingers crossed)**

In [71]:
#set value of levenshtein distance threshold (4 means only distances of 3 and lower would be considered)
lev_dist_threshold = 4

#loop through each yelp bizID and find the restaurant with closest lev distance (currently matches using name only)
bizrevs['lev_dist'] = lev_dist_threshold
bizrevs['insp_bizID'] = ""
for i in range(len(bizrevs['bizID'])):
    for x in range(len(insp['insp_bizID'])):
        dist = lev(str(bizrevs['name'][i]),str(insp['insp_biz_name'][x]))
        if dist < bizrevs['lev_dist'][i]:
            bizrevs['lev_dist'][i] = dist
            bizrevs['insp_bizID'][i] = insp['insp_bizID'][x]

df = pd.merge(bizrevs,insp,on='insp_bizID', how='left')            

df.head()

,bizID,name,address,reviews,checkins,stars_1,stars_2,stars_3,stars_4,stars_5,...,insp_bizID,insp_biz_name,insp_biz_address,insp_date,last_inspection,insp_count_minor,insp_count_significant,insp_count_crucial,insp_count_na,insp_total_count_cs
0,--SrzpvFLwP_YFwB_Cetow,NaN,NaN,"""Out of all the Keung Kee locations, my family...",0,1,2,4,10,4,...,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0_MI3uzOP0MMPNSfj5zB-g,NaN,NaN,"""Can't say they've kept up the good work. Pri...",0,1,1,0,0,0,...,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4HeS4zlbf-IbwEItbrISlw,NaN,NaN,"""This is a welcome addition to the neighborhoo...",0,2,1,0,6,2,...,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,9iIztpy5MJuSFrLgK1HIcw,NaN,NaN,"""I'll never understand the draw to Tim Horton'...",0,0,2,2,2,0,...,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Cfj526DoysI9UvojIrRsUw,NaN,NaN,"""$3.99 all day specials.. no judgement please....",0,0,1,2,0,0,...,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [75]:
#import toronto inspection data
cur.execute("SELECT * FROM public.toronto_inspections LIMIT 50")
insp = pd.DataFrame(cur.fetchall())
insp.columns = ['insp_bizID','insp_biz_name','insp_biz_address','insp_date','last_inspection','insp_count_minor','insp_count_significant','insp_count_crucial','insp_count_na','insp_total_count_cs']
insp.head()

,insp_bizID,insp_biz_name,insp_biz_address,insp_date,last_inspection,insp_count_minor,insp_count_significant,insp_count_crucial,insp_count_na,insp_total_count_cs
0,10187087,METROPOLIS BAKESHOP,2 BLOOR ST W,3/17/17,10/18/16,0,0,0,0,0
1,10289695,MILAN'S PIZZERIA & WINGS,1792 WESTON RD,11/9/15,11/2/15,0,0,0,0,0
2,10355463,PIZZA PASTA WAYS,3300 BLOOR ST W,12/15/16,10/7/15,2,0,0,0,0
3,10453689,TRINITY - ST. PAUL'S UNITED CHURCH,427 BLOOR ST W,2/18/16,12/21/15,0,0,0,0,0
4,10503134,INTERNATIONAL NEWS SHEPPARD,45 SHEPPARD AVE E,1/31/17,1/31/17,1,0,0,0,0


**following section is to normalize addresses. a package is available but doing it manually seems easier / good enough. package:** https://github.com/pnpnpn/street-address 

In [72]:
#normalizes addresses
biz['address'] = [addr.replace('Street','St').replace('Boulevard','Blvd').replace('Avenue','Ave').replace('Road','Rd')
        .replace('North','N').replace('West','W').replace('South','S').replace('East','E') for addr in biz['address']]

#we should also think about removing pre and suffixes, like this example (insp data does not seem to have units etc):
biz['address'][37]

'1176 Danforth Ave E'